<a href="https://colab.research.google.com/github/Arumat546/nico_cls/blob/main/%E3%83%86%E3%82%AD%E3%82%B9%E3%83%88%E5%89%8D%E5%87%A6%E7%90%86.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%cd /content/drive/My Drive/Google Colab/niconico_dataset_add_smid

/content


In [ ]:
#実行
import pandas as pd
import csv

df = pd.read_table("seizi.tsv", encoding='utf-8')
#print(df)
#print(df['content'][8])

#前処理
df['content'] = df['content'].map(clean_text)
df['content'] = df['content'].map(normalize)

#3文字以下の文章の除去
df['content'] = df['content'].astype('str')
mask = (df['content'].str.len() > 3)
df = df.loc[mask]
print(df)


df.to_csv("seizi8000.tsv", sep='\t', index=None, encoding='utf-8')
#print(df)

                                        content  category
0                           0年以内に日本の風景はガラリと変わるぞ         0
1           人は常は無限に足りないので外国人を入れても解消されませんだからいらない         0
2                               あべぴょん信者冷えてるかぁ~w         0
3                       これで日本は移民大国として世界0位に躍り出るぞ         0
4                    その移民が年間0万人の日本人を殺す未来しか見えないな         0
...                                         ...       ...
38423               康はronに裏切られましたが政治ですから甘すぎませんか         0
38424                  弱みを捕まれるような行動を取っていると罠にかかる         0
38425  新幹線で東京-新横浜は約0km弱 大体東京-三島が約0kmで東京-掛川が約0km         0
38426                               青山繁晴先生と日蓮聖人         0
38427                                 三方ヶ原の戦いじゃ         0

[38088 rows x 2 columns]


## **記号除去**

In [ ]:
#記号除去
import string
def clean_text(text):
    #replaced_text = '\n'.join(s.strip() for s in text.splitlines()[2:] if s != '')  # skip header by [2:]
    replaced_text = text.lower()
    replaced_text = re.sub(r'[【】]', '', text)       # 【】の除去
    replaced_text = re.sub(r'[（）()]', '', replaced_text)     # （）の除去
    replaced_text = re.sub(r'[［］\[\]]', '', replaced_text)   # ［］の除去
    replaced_text = re.sub(r'[「」｢｣『』]', '', replaced_text)     # 「」の除去
    replaced_text = re.sub(r'[{}｛｝]', '', replaced_text)    #  {}の除去
    replaced_text = re.sub(r'[；;＾￥：:、。・￥!！"#$%&=~”＃＄％＆’|*+<>/?？_\.※‥…→←↑↓]', '', replaced_text)    #  その他記号の除去   
    replaced_text = re.sub(r'https?:\/\/.*?[\r\n ]', '', replaced_text)  # URLの除去
    replaced_text = re.sub(r'　', '', replaced_text)  # 全角空白の除去 
    return replaced_text    


## **基本前処理**

In [ ]:
# -*- coding: utf-8 -*-
import re
import unicodedata
import nltk
from nltk.corpus import wordnet

#まとめて処理
def normalize(text):
    normalized_text = normalize_unicode(text)
    normalized_text = normalize_number(normalized_text)
    normalized_text = lower_text(normalized_text)
    normalized_text = reduce_seq(normalized_text)
    normalized_text = reduce_w(normalized_text)
    return normalized_text


#大文字→小文字
def lower_text(text):
    return text.lower()

#半角全角統一
def normalize_unicode(text, form='NFKC'):
    normalized_text = unicodedata.normalize(form, text)
    return normalized_text

#数字列の置換
def normalize_number(text):
    """
    pattern = r'\d+'
    replacer = re.compile(pattern)
    result = replacer.sub('0', text)
    """
    # 連続した数字を0で置換
    replaced_text = re.sub(r'\d+', '0', text)
    return replaced_text

#3文字以上連続を2文字に変換
def reduce_seq(text):

  result = re.sub(r'(.)\1{2,}',r'\1', text)
  return result
#二文字の除草
def reduce_w(text):

  result = re.sub(r'w{2}',r'', text)
  return result

In [ ]:
text='ここはこここはここここはこここここww'
ntext=reduce_w(text)
print(ntext)

ここはこここはここここはこここここ


## **同義語の統一**

In [ ]:
import re
import unicodedata
import nltk
from nltk.corpus import wordnet

def lemmatize_term(term, pos=None):
    if pos is None:
        synsets = wordnet.synsets(term)
        if not synsets:
            return term
        pos = synsets[0].pos()
        if pos == wordnet.ADJ_SAT:
            pos = wordnet.ADJ
    return nltk.WordNetLemmatizer().lemmatize(term, pos=pos)

## **絵文字と顔文字除去**


## **ストップワード除去**

In [ ]:
# -*- coding: utf-8 -*-
import os
import urllib.request
from collections import Counter

from gensim import corpora


def maybe_download(path):
    url = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
    if os.path.exists(path):
        print('File already exists.')
    else:
        print('Downloading...')
        # Download the file from `url` and save it locally under `file_name`:
        urllib.request.urlretrieve(url, path)


def create_dictionary(texts):
    dictionary = corpora.Dictionary(texts)
    return dictionary


def remove_stopwords(words, stopwords):
    words = [word for word in words if word not in stopwords]
    return words


def most_common(docs, n=100):
    fdist = Counter()
    for doc in docs:
        for word in doc:
            fdist[word] += 1
    common_words = {word for word, freq in fdist.most_common(n)}
    print('{}/{}'.format(n, len(fdist)))
    return common_words


def get_stop_words(docs, n=100, min_freq=1):
    fdist = Counter()
    for doc in docs:
        for word in doc:
            fdist[word] += 1
    common_words = {word for word, freq in fdist.most_common(n)}
    rare_words = {word for word, freq in fdist.items() if freq <= min_freq}
    stopwords = common_words.union(rare_words)
    print('{}/{}'.format(len(stopwords), len(fdist)))
    return stopwords

## **試行**

In [ ]:
text='私はわたし  であるが1235555toTOsonyは；＾￥#$%&=~|*+<>/?_\.「」[]{}()（）ｿﾆｰソニーｋk9ああああaaaa'
ntext=clean_text(text)
print(ntext)

私はわたし  であるが1235555totosonyは\ｿﾆｰソニーｋk9ああああaaaa
